In [1]:
%%writefile requirements.txt
-f --extra-index-url https://download.pytorch.org/whl/cu116
torch
torchvision
numpy
transformers
git+https://github.com/huggingface/diffusers
gradio
accelerate

Overwriting requirements.txt


In [2]:
!pip3 install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: --extra-index-url
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-fqvvfl0e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-fqvvfl0e
  Resolved https://github.com/huggingface/diffusers to commit 69caa9647201b08119288ab1df6574552635b37e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import PIL
from PIL import Image, ImageOps
import math
import gradio as gr
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipe.scheduler.config)


def generate(
    input_image: Image.Image,
    instruction: str,
):
    width, height = input_image.size
    factor = 512 / max(width, height)
    factor = math.ceil(min(width, height) * factor /
                       64) * 64 / min(width, height)
    width = int((width * factor) // 64) * 64
    height = int((height * factor) // 64) * 64
    input_image = ImageOps.fit(
        input_image, (width, height), method=Image.LANCZOS)

    if instruction == "":
        return input_image

    images = pipe(instruction, image=input_image, num_inference_steps=10,
                  image_guidance_scale=1).images[0]
    return images


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=3):
            instruction = gr.Textbox(
                lines=1, label="Instructions", interactive=True)

    with gr.Row():
        input_image = gr.Image(label="Input Image",
                               type="pil", interactive=True)
        edited_image = gr.Image(
            label=f"Edited Image", type="pil", interactive=False)
        input_image.style(height=512, width=512)
        edited_image.style(height=512, width=512)

    with gr.Row():
        with gr.Column(scale=1, min_width=100):
            generate_button = gr.Button("Generate")
    generate_button.click(
        fn=generate, inputs=[input_image, instruction], outputs=[edited_image])

demo.queue(concurrency_count=1)
demo.launch(share=True, debug=True)


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6226cc83-523a-458a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6226cc83-523a-458a.gradio.live
